In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf


from modules.utils import change_X_data_2, create_y_test_file

In [ ]:
model = tf.keras.models.load_model("../models/second_model_4.h5")

In [ ]:
x_test = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_test.csv")
x_test_2 = change_X_data_2(x_test)
x_test_3 = np.array([np.array(row) for row in x_test_2["column 1"]])

In [ ]:
y_test_result = create_y_test_file(x_test_3, model)


y_test_result.head()

In [ ]:
y_test_result.to_csv('/Users/elouan/Data/Data_ENSxCFM/y_test_6(not submitted).csv', index=False)